In [52]:
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix, hstack
from sklearn.naive_bayes import MultinomialNB
import ast

In [53]:
data = pd.read_csv("../data_chunks/out0.csv")

In [55]:
data.columns

Index(['Unnamed: 0', 'no', 'id', 'spam', 'source', 'tweet', 'created_at',
       'followers_count', 'friends_count', 'hashtags', 'symbols', 'text',
       'urls', 'user_description', 'user_id', 'user_location', 'user_name',
       'verified'],
      dtype='object')

In [56]:
data.drop(['Unnamed: 0',"no","tweet","symbols"] ,axis=1, inplace=True)

In [59]:
data[data['spam'].isnull()]

,id,spam,source,created_at,followers_count,friends_count,hashtags,text,urls,user_description,user_id,user_location,user_name,verified


In [62]:
y = data['spam']

In [63]:
y.value_counts()

 0    497
 1     59
-1      2
Name: spam, dtype: int64

In [64]:
X= data["text"].values
y = data['spam'].values

In [92]:
l = ast.literal_eval(data["hashtags"][7])
print(l[0]["text"])


In [111]:
def get_hashtags(tag):
    ans = []
    tag = ast.literal_eval(tag)

    for i in tag:
        if len(tag) > 0:
            ans.append(i["text"])
    ans  = ",".join(str(x) for x in ans)
    return ans

In [113]:
data["hashtags"] = data["hashtags"].apply(get_hashtags)

In [121]:
H = data["hashtags"].values

In [123]:
tfidf_hash = TfidfVectorizer(encoding='latin-1',binary=True, ngram_range=(1,1), min_df=2, use_idf=True, stop_words='english')


In [127]:
H_vec = tfidf_hash.fit_transform(H)


In [154]:
data["verified_num"] = data["verified"].apply(lambda x: 1 if x == 'True' else 0)

In [145]:
data[data["user_name"] == "."]

,id,spam,source,created_at,followers_count,friends_count,hashtags,text,urls,user_description,user_id,user_location,user_name,verified
67,329850900619341824,0,5,Thu May 02 06:52:37 +0000 2013,1604,324,,while no one knows 80 percent of the shit you'...,[],ㄱ,31466190,"Murrieta, CA",.,False
242,329850932588318720,0,5,Thu May 02 06:52:45 +0000 2013,1292,546,,fall in love with minwoo(boyfriend)!!!!!,[],[ i'm gonna cherish you forever ],1016035394,"multifandom , bias | chinaline",.,False
253,329850934471565312,1,6,Thu May 02 06:52:45 +0000 2013,297,282,,@giannaa_rose that unfollower can go fuck em s...,[],NaN,515273018,NaN,.,False
415,329850968172793856,0,5,Thu May 02 06:52:53 +0000 2013,1060,362,,@KatieandthatEGO you'd think after moving acro...,[],NaN,64097067,NaN,.,False
423,329850971922526208,0,5,Thu May 02 06:52:54 +0000 2013,1106,0,,Writing music.,[],NaN,271139177,.,.,False
482,329850984803213312,0,6,Thu May 02 06:52:57 +0000 2013,39,178,,@Dimplesz567 baby I love uuuuuu and wen I slid...,[],NaN,1118384142,NaN,.,False


In [38]:
tfidf = TfidfVectorizer(encoding='latin-1',binary=True, ngram_range=(1,2), min_df=5, use_idf=True, stop_words='english')

In [129]:
X_vec = tfidf.fit_transform(X)

In [239]:
X_vec.shape


(558, 132)

### columns to use

In [49]:
# X_vec , H_vec, followers_count , friends_count , user_id , verified_num

(558, 123)

In [231]:
A = coo_matrix(X_vec)
B = coo_matrix(H_vec)
C = coo_matrix(data["followers_count"].values)
D = coo_matrix(data["friends_count"].values)
E = coo_matrix(data["user_id"].values)
F = coo_matrix(data["verified_num"].values)


In [232]:
tf_vectors = hstack([A,B,C.T,D.T,E.T,F.T]).toarray()


In [233]:
tf_vectors.shape


(558, 155)

In [235]:
nb_clf= MultinomialNB()


In [236]:
nb_clf.fit(tf_vectors,Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [237]:
nb_clf.score(tf_vectors,Y)

0.45519713261648748